In [1]:
from custom_dataset import custom_collate, Dataset_Fusion_MutationList_Binary
from fusion_model import *

# LOAD DATA
# canonical_mut_embeddings_esm2 = np.load('../aa/canonical_mut_embeddings_esm2.npy')
canonical_ave_embeddings_esm2 = np.load('../aa/canonical_mut_average_embeddings_esm2.npy')
data_dir = '../labeled_data/'
labeled_data = os.listdir(data_dir)
for ni,i in sorted(zip(labeled_data,range(len(labeled_data)))):print(i,'\t',ni)

data_emb_filename = labeled_data[29]
print('\n',data_emb_filename)
data_emb = pd.read_csv(data_dir+data_emb_filename)
emb_label_map = pickleLoad(data_dir+'/label_mappings/'+'label_mapping_'+data_emb_filename.strip('.csv')+ '.pkl')

data_bin_filename = labeled_data[0]
print('\n',data_bin_filename)
data_bin = pd.read_csv(data_dir+data_bin_filename)
bin_label_map = pickleLoad(data_dir+'/label_mappings/'+'label_mapping_'+data_bin_filename.strip('.csv')+ '.pkl')

labels = torch.tensor(data_emb['int_label'].values,dtype=torch.long)
nlabels = len(data_emb['int_label'].unique())
device = 'cuda:1'

# Create dataset
# dataset = Dataset_Fusion_MutationList_Binary(data_bin, data_emb, bin_label_map, emb_label_map, canonical_mut_embeddings_esm2,device)
dataset = Dataset_Fusion_MutationList_Binary(data_bin, data_emb, bin_label_map, emb_label_map, canonical_ave_embeddings_esm2,device)

# Create DataLoader
# dataloader = DataLoader(dataset, batch_size=100, shuffle=False, collate_fn=custom_collate)

## TEST/TRAIN SPLIT
test_size = .2
random_state = 42
batch_size = 1
indices = list(range(len(dataset)))

train_indices, test_indices = train_test_split(
    indices, 
    test_size=test_size, 
    random_state=random_state
)

train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

26 	 BINARYdata_CANCER_TYPE_0MinMutations_1696MinCancerType.csv
16 	 BINARYdata_CANCER_TYPE_0MinMutations_169MinCancerType.csv
0 	 BINARYdata_CANCER_TYPE_3MinMutations_1696MinCancerType.csv
23 	 BINARYdata_CANCER_TYPE_3MinMutations_169MinCancerType.csv
8 	 BINARYdata_CANCER_TYPE_DETAILED_0MinMutations_1696MinCancerType.csv
18 	 BINARYdata_CANCER_TYPE_DETAILED_0MinMutations_169MinCancerType.csv
6 	 BINARYdata_CANCER_TYPE_DETAILED_3MinMutations_1696MinCancerType.csv
22 	 BINARYdata_CANCER_TYPE_DETAILED_3MinMutations_169MinCancerType.csv
28 	 canonical_ref_barcode_refIdxs_map.pkl
15 	 data_CANCER_TYPE_0MinMutations_1696MinCancerType.csv
17 	 data_CANCER_TYPE_0MinMutations_169MinCancerType.csv
29 	 data_CANCER_TYPE_3MinMutations_1696MinCancerType.csv
12 	 data_CANCER_TYPE_3MinMutations_169MinCancerType.csv
7 	 data_CANCER_TYPE_DETAILED_0MinMutations_1696MinCancerType.csv
20 	 data_CANCER_TYPE_DETAILED_0MinMutations_169MinCancerType.csv
1 	 data_CANCER_TYPE_DETAILED_3MinMutations_1696MinCan

In [2]:
# majority classifier
print(len(data_emb['int_label'].unique()))
sorted(data_emb['int_label'].value_counts(),reverse=True)[0]/len(data_emb['int_label'])

20


0.18109837773478196

In [3]:
import torch.optim as optim
from tqdm import tqdm

class Config:
    n_layer: int = 3
    input_dim: int = 640
    dropout: float = 0.0
    bias: bool = False
    n_labels: int = 17
    pooling : str = 'mean'
    binary_dim: int = 1448
    norm_fn: nn.Module = nn.LayerNorm

print('n labels:',nlabels)
config = Config()
config.n_labels = nlabels

model = Classifier(config)
model.to(device)

num_epochs = 5
learning_rate = 0.001

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    with tqdm(enumerate(train_loader), total=len(train_loader),desc='TRAINING') as pbar:
        for batch_idx, (data1, data2, target) in pbar:
            optimizer.zero_grad()
            output = model(data1,data2)
            # assert False
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            pbar.set_postfix({'Epoch':f'{epoch+1}/{num_epochs}, Loss: {loss.item():.4f}'})
            if batch_idx % 20000 == 0:
                print('')

        # Evaluation
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for data1,data2, target in tqdm(test_loader,desc='TESTING'):
                output = model(data1,data2)
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        accuracy = 100 * correct / total
        print(f'Test Accuracy: {accuracy:.2f}%, ({correct} of {total})')

n labels: 20


TRAINING:   0%|          | 18/80628 [00:00<16:20, 82.18it/s, Epoch=1/5, Loss: 22.8993] 

TRAINING:  25%|██▍       | 20021/80628 [02:24<07:23, 136.77it/s, Epoch=1/5, Loss: 8.6600]  

TRAINING:  50%|████▉     | 40017/80628 [04:51<05:09, 131.38it/s, Epoch=1/5, Loss: 0.5787]  

TRAINING:  74%|███████▍  | 60024/80628 [07:14<02:24, 142.84it/s, Epoch=1/5, Loss: 0.0645]  

...

TRAINING:  50%|████▉     | 40021/80628 [04:43<05:39, 119.49it/s, Epoch=5/5, Loss: 0.0000] 

TRAINING:  74%|███████▍  | 60014/80628 [07:39<03:03, 112.33it/s, Epoch=5/5, Loss: 0.2449] 

TRAINING:  99%|█████████▉| 80018/80628 [10:35<00:05, 113.25it/s, Epoch=5/5, Loss: 2.4820] 

TESTING: 100%|██████████| 20157/20157 [00:20<00:00, 985.36it/s]

Test Accuracy: 49.44%, (9966 of 20157)
